In [1]:
import os
import numpy as np
import pandas as pd
import polars as pl
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from pymer4.models import glmer
from statsmodels.graphics.gofplots import qqplot
from plotly.subplots import make_subplots
import matplotlib
import pymer4.tidystats as ts
%matplotlib agg

pio.templates.default = "plotly_white"

STAIN_NAMES = dict(I2KI='Iodine-stained', PTA='PTA-stained')
TISSUE_NAMES = dict(Muscle='Muscle-serosal', Mucosa='Mucosal-submucosal')
COLOR_SCALE = ['#6C8EBF', '#82B366', '#C6C013', '#E58110', '#B85450', '#9673A6', '#666666', '#A65628', '#E47E77', '#8F2D56', '#73937E', '#59548C']

def stat_diagnostics(fitted_values, residuals, group):
    # group_enc = group.to_arrow().indices
    
    # stat, p = stats.shapiro(residuals)
    # print(f"Residuals: W={stat:.3f}, p={p:.3f} {'✅ Normal' if p > 0.05 else '❌ Not normal'}")
    fig = make_subplots(rows=2, cols=2)

    # fig.add_trace(
    #     go.Scatter(x=fitted_values, y=residuals, mode='markers', marker_color=group_enc, marker_colorscale=px.colors.qualitative.Dark24, text=group),
    #     row=1, col=1, 
    # )
    fig.add_trace(
        go.Scatter(x=fitted_values, y=residuals, mode='markers', marker_color=COLOR_SCALE[0], text=group),
        row=1, col=1, 
    )

    qqplot_data = qqplot(residuals, line='s').gca().lines

    fig.add_scatter(x=qqplot_data[0].get_xdata(), y=qqplot_data[0].get_ydata(), mode='markers', marker_size=10, marker_color=COLOR_SCALE[0], row=1, col=2,)
    fig.add_scatter(x=qqplot_data[1].get_xdata(), y=qqplot_data[1].get_ydata(), mode='lines', marker_line_width=2, marker_color=COLOR_SCALE[4], row=1, col=2)
    del qqplot_data
    sqrt_abs_res = np.sqrt(np.abs(residuals))
    # fig.add_trace(
    #     go.Scatter(x=fitted_values, y=sqrt_abs_res, mode='markers', marker_color=group_enc, text=group, marker_colorscale=px.colors.qualitative.Dark24),
    #     row=2, col=1,
    # )

    fig.add_trace(
        go.Scatter(x=fitted_values, y=sqrt_abs_res, mode='markers', marker_color=COLOR_SCALE[0], text=group),
        row=2, col=1,
    )

    fig.add_histogram(x=residuals, histnorm='probability', marker_color=COLOR_SCALE[0], row=2, col=2)

    # fig = px.scatter(vis, x="Fitted values", y="Residuals", color="Experiment", color_discrete_sequence=px.colors.qualitative.Dark24)
    fig.update_traces(marker_size=10, row=1, col=1)
    fig.update_traces(marker_size=10, row=2, col=1)
    fig.update_layout(width=800, height=500, showlegend=False, margin=dict(t=5, b=5, l=10, r=10))

    # Update xaxis properties
    fig.update_xaxes(title_text="Fitted", row=1, col=1)
    fig.update_xaxes(title_text="Theoretical Quantiles", row=1, col=2)
    fig.update_xaxes(title_text="Fitted", row=2, col=1)
    fig.update_xaxes(title_text="Residuals", row=2, col=2)

    # Update yaxis properties
    fig.update_yaxes(title_text="Residuals", row=1, col=1)
    fig.update_yaxes(title_text="Sample Quantiles", row=1, col=2)
    fig.update_yaxes(title_text="Sqrt( Abs( Residuals ) )", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=2)

    fig.show()

    
filepaths = {}
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        key = filename.split('_')[-1].split('.')[0]
        filepaths[key] = os.path.join(dirname, filename)

print(filepaths)
output_path = "."

{'measures': './data/quantitative_quality_measures.csv', 'metadata': './data/experiment_metadata.csv'}


In [2]:
df_metadata = pd.read_csv(filepaths["metadata"])
df_measures = pd.read_csv(filepaths["measures"])

display(df_metadata.head())
display(df_measures.head())

,Title,Male,Age (weeks),Stain,Resolution (um),Collection Date,Stain start,Imaging - Stained,Imaging - Fixed
0,Rat 01,Yes,10,I2KI,10.03,"May 25, 2022","June 15, 2022","July 6, 2022","June 2, 2022"
1,Rat 02,No,10,I2KI,10.03,"June 29, 2022","September 12, 2022","October 7, 2022",NaN
2,Rat 03,Yes,10,PTA,7.72,"March 29, 2023","April 19, 2023","May 23, 2023",NaN
3,Rat 04,Yes,10,PTA,7.72,"March 30, 2023","April 19, 2023","June 8, 2023",NaN
4,Rat 05,Yes,8,PTA,7.99,"April 19, 2023","June 12, 2023","September 13, 2023",NaN


,Experiment,Tissue,ROI,Area,Mean,StdDev,Min,Max,Median,SNR
0,RT01,Muscle,GC_Antrum,0.1,79.056,9.768,42,105,79,8.093366
1,RT01,Muscle,GC_Pylorus,0.1,73.901,10.629,32,98,76,6.952771
2,RT01,Muscle,LC_Pylorus,0.1,93.076,9.392,62,119,93,9.910136
3,RT01,Muscle,LC_LES,0.1,101.922,10.790,65,138,102,9.445968
4,RT01,Muscle,GC_LES,0.1,113.596,12.897,77,154,113,8.807940


In [3]:
df_metadata = df_metadata[['Title', 'Stain']]
df_metadata.rename({'Title': 'Experiment'}, axis=1, inplace=True)
df_metadata['Experiment'] = df_metadata['Experiment'].str.replace('at ', 'T')
df_metadata['Stain'] = df_metadata['Stain'].replace(STAIN_NAMES)
df_metadata.head()

,Experiment,Stain
0,RT01,Iodine-stained
1,RT02,Iodine-stained
2,RT03,PTA-stained
3,RT04,PTA-stained
4,RT05,PTA-stained


In [4]:
df_measures = pd.merge(df_measures, df_metadata, on="Experiment", how="left")
df_measures.head()

,Experiment,Tissue,ROI,Area,Mean,StdDev,Min,Max,Median,SNR,Stain
0,RT01,Muscle,GC_Antrum,0.1,79.056,9.768,42,105,79,8.093366,Iodine-stained
1,RT01,Muscle,GC_Pylorus,0.1,73.901,10.629,32,98,76,6.952771,Iodine-stained
2,RT01,Muscle,LC_Pylorus,0.1,93.076,9.392,62,119,93,9.910136,Iodine-stained
3,RT01,Muscle,LC_LES,0.1,101.922,10.790,65,138,102,9.445968,Iodine-stained
4,RT01,Muscle,GC_LES,0.1,113.596,12.897,77,154,113,8.807940,Iodine-stained


In [5]:
# Assuming df_measures is your input DataFrame
data = df_measures[["Experiment", "Stain", "ROI", "Tissue", "Mean", "StdDev"]]

# Pivot to get 'muscle' and 'mucosa' stats in columns
df_cnr = data.pivot_table(index=["Experiment", "Stain", "ROI"], columns="Tissue", values=["Mean", "StdDev"])

# Flatten the multi-level columns
df_cnr.columns = ['_'.join(col).strip().lower() for col in df_cnr.columns.values]
df_cnr.reset_index(inplace=True)
display(df_cnr.head(10))

# Compute CNR
df_cnr['CNR'] = np.abs(df_cnr['mean_muscle'] - df_cnr['mean_mucosa']) / np.sqrt(df_cnr['stddev_muscle']**2 + df_cnr['stddev_mucosa']**2)

df_cnr = df_cnr.reset_index()[["Experiment", "Stain", "ROI", "CNR"]]
df_cnr.head()

,Experiment,Stain,ROI,mean_mucosa,mean_muscle,stddev_mucosa,stddev_muscle
0,RT01,Iodine-stained,GC_Antrum,72.612,79.056,8.473,9.768
1,RT01,Iodine-stained,GC_Corpus,85.331,91.738,8.788,10.509
2,RT01,Iodine-stained,GC_Fundus1,81.945,83.246,7.481,9.304
3,RT01,Iodine-stained,GC_Fundus2,67.023,70.690,5.526,5.302
4,RT01,Iodine-stained,GC_LES,112.101,113.596,13.957,12.897
5,RT01,Iodine-stained,GC_Pylorus,70.499,73.901,9.478,10.629
6,RT01,Iodine-stained,LC_Antrum,81.169,87.435,10.681,10.718
7,RT01,Iodine-stained,LC_Corpus,93.142,109.951,10.162,10.288
8,RT01,Iodine-stained,LC_LES,99.382,101.922,12.529,10.790
9,RT01,Iodine-stained,LC_Pylorus,82.348,93.076,10.631,9.392


,Experiment,Stain,ROI,CNR
0,RT01,Iodine-stained,GC_Antrum,0.498345
1,RT01,Iodine-stained,GC_Corpus,0.467692
2,RT01,Iodine-stained,GC_Fundus1,0.108974
3,RT01,Iodine-stained,GC_Fundus2,0.478834
4,RT01,Iodine-stained,GC_LES,0.078670


In [6]:
df_measures['Tissue'] = df_measures['Tissue'].replace(TISSUE_NAMES)

# Ensure categorical types for statsmodels
df_measures['Experiment'] = pd.Categorical(df_measures['Experiment'])
df_measures['Stain'] = pd.Categorical(df_measures['Stain'])
df_measures['Tissue'] = pd.Categorical(df_measures['Tissue'])
df_measures['ROI'] = pd.Categorical(df_measures['ROI'])

df_measures.head()

,Experiment,Tissue,ROI,Area,Mean,StdDev,Min,Max,Median,SNR,Stain
0,RT01,Muscle-serosal,GC_Antrum,0.1,79.056,9.768,42,105,79,8.093366,Iodine-stained
1,RT01,Muscle-serosal,GC_Pylorus,0.1,73.901,10.629,32,98,76,6.952771,Iodine-stained
2,RT01,Muscle-serosal,LC_Pylorus,0.1,93.076,9.392,62,119,93,9.910136,Iodine-stained
3,RT01,Muscle-serosal,LC_LES,0.1,101.922,10.790,65,138,102,9.445968,Iodine-stained
4,RT01,Muscle-serosal,GC_LES,0.1,113.596,12.897,77,154,113,8.807940,Iodine-stained


In [7]:
pl_df_measures = pl.DataFrame(df_measures)
print(pl_df_measures.columns)
display(pl_df_measures.head())

['Experiment', 'Tissue', 'ROI', 'Area', 'Mean', 'StdDev', 'Min', 'Max', 'Median', 'SNR', 'Stain']


Experiment,Tissue,ROI,Area,Mean,StdDev,Min,Max,Median,SNR,Stain
cat,cat,cat,f64,f64,f64,i64,i64,i64,f64,cat
"""RT01""","""Muscle-serosal""","""GC_Antrum""",0.1,79.056,9.768,42,105,79,8.093366,"""Iodine-stained"""
"""RT01""","""Muscle-serosal""","""GC_Pylorus""",0.1,73.901,10.629,32,98,76,6.952771,"""Iodine-stained"""
"""RT01""","""Muscle-serosal""","""LC_Pylorus""",0.1,93.076,9.392,62,119,93,9.910136,"""Iodine-stained"""
"""RT01""","""Muscle-serosal""","""LC_LES""",0.1,101.922,10.79,65,138,102,9.445968,"""Iodine-stained"""
"""RT01""","""Muscle-serosal""","""GC_LES""",0.1,113.596,12.897,77,154,113,8.80794,"""Iodine-stained"""


In [8]:
glmm_snr_snt = glmer('SNR ~ Stain * Tissue + (1|Experiment) + (1|ROI)', data=pl_df_measures, family='Gamma', link='log')
glmm_snr_snt.set_factors(['Stain', 'Tissue'])

# Convert estimates to odds
display(glmm_snr_snt.fit(exponentiate=True, summary=True, conf_method='boot'))
display(glmm_snr_snt.result_fit_stats)
stat_diagnostics(glmm_snr_snt.data['fitted'], glmm_snr_snt.data['resid'], glmm_snr_snt.data['Experiment'])

GT(_tbl_data=shape: (9, 10)
┌────────────────┬─────────────────┬──────────┬───────────┬───┬───────────┬──────┬─────────┬───────┐
│ rfx            ┆ param           ┆ estimate ┆ conf_low  ┆ … ┆ z_stat    ┆ df   ┆ p_value ┆ stars │
│ ---            ┆ ---             ┆ ---      ┆ ---       ┆   ┆ ---       ┆ ---  ┆ ---     ┆ ---   │
│ str            ┆ str             ┆ f64      ┆ f64       ┆   ┆ f64       ┆ i64  ┆ str     ┆ str   │
╞════════════════╪═════════════════╪══════════╪═══════════╪═══╪═══════════╪══════╪═════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)     ┆ 0.096793 ┆ 0.141415  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ ROI-sd         ┆ (Intercept)     ┆ 0.063451 ┆ 0.102447  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Residual-sd    ┆ Observation     ┆ 0.203774 ┆ 0.21708   ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ null            ┆ null     ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Fixed Effects: ┆ null            ┆ null     ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ (Intercept)     ┆ 9.911816 ┆ 1.772682  ┆ … ┆ 18.265743 ┆ 253  ┆ <.001   ┆ ***   │
│ null           ┆ StainPTA-staine ┆ 1.35851  ┆ -0.231546 ┆ … ┆ 2.303716  ┆ 253  ┆ 0.02205 ┆ *     │
│                ┆ d               ┆          ┆           ┆   ┆           ┆      ┆         ┆       │
│ null           ┆ TissueMuscle-se ┆ 1.040002 ┆ -0.051307 ┆ … ┆ 0.923888  ┆ 253  ┆ 0.3564  ┆       │
│                ┆ rosal           ┆          ┆           ┆   ┆           ┆      ┆         ┆       │
│ null           ┆ StainPTA-staine ┆ 0.987811 ┆ -0.119993 ┆ … ┆ -0.240388 ┆ 253  ┆ 0.8102  ┆       │
│                ┆ d:TissueMuscle- ┆          ┆           ┆   ┆           ┆      ┆         ┆       │
│                ┆ …               ┆          ┆           ┆   ┆           ┆      ┆         ┆       │
└────────────────┴─────────────────┴──────────┴───────────┴───┴───────────┴──────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e74c6ebc230>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e74c6ed5730>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(SNR~Stain*Tissue+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *260*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-609*  \n            AIC: *1232* | BIC: *1257*  \n            Residual error: *0.204*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, col

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
1232.691493,1233.135937,1257.616264,0.247689,0.447243,0.265255,2.472008,0.203774,9.398417,253,-609.345746,260,0.203774


In [9]:
glmm_snr_snt.conf_method

In [10]:
glmm_snr_sot = glmer('SNR ~ Stain + Tissue + (1|Experiment) + (1|ROI)', data=pl_df_measures, family='Gamma', link='log')
glmm_snr_sot.set_factors(['Stain', 'Tissue'])

# Convert estimates to odds
display(glmm_snr_sot.fit(exponentiate=True, summary=True, conf_method='boot'))

display(glmm_snr_sot.result_fit_stats)
stat_diagnostics(glmm_snr_sot.data['fitted'], glmm_snr_sot.data['resid'], glmm_snr_sot.data['Experiment'])

GT(_tbl_data=shape: (8, 10)
┌────────────────┬─────────────────┬──────────┬───────────┬───┬───────────┬──────┬─────────┬───────┐
│ rfx            ┆ param           ┆ estimate ┆ conf_low  ┆ … ┆ z_stat    ┆ df   ┆ p_value ┆ stars │
│ ---            ┆ ---             ┆ ---      ┆ ---       ┆   ┆ ---       ┆ ---  ┆ ---     ┆ ---   │
│ str            ┆ str             ┆ f64      ┆ f64       ┆   ┆ f64       ┆ i64  ┆ str     ┆ str   │
╞════════════════╪═════════════════╪══════════╪═══════════╪═══╪═══════════╪══════╪═════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)     ┆ 0.096805 ┆ 0.140871  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ ROI-sd         ┆ (Intercept)     ┆ 0.063434 ┆ 0.106411  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Residual-sd    ┆ Observation     ┆ 0.203785 ┆ 0.216015  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ null            ┆ null     ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Fixed Effects: ┆ null            ┆ null     ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ (Intercept)     ┆ 9.953884 ┆ 1.771778  ┆ … ┆ 18.477378 ┆ 254  ┆ <.001   ┆ ***   │
│ null           ┆ StainPTA-staine ┆ 1.350221 ┆ -0.230465 ┆ … ┆ 2.299355  ┆ 254  ┆ 0.0223  ┆ *     │
│                ┆ d               ┆          ┆           ┆   ┆           ┆      ┆         ┆       │
│ null           ┆ TissueMuscle-se ┆ 1.031215 ┆ -0.023521 ┆ … ┆ 1.302459  ┆ 254  ┆ 0.1939  ┆       │
│                ┆ rosal           ┆          ┆           ┆   ┆           ┆      ┆         ┆       │
└────────────────┴─────────────────┴──────────┴───────────┴───┴───────────┴──────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e75b053c080>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e75b0660fb0>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(SNR~Stain+Tissue+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *260*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-609*  \n            AIC: *1230* | BIC: *1252*  \n            Residual error: *0.204*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=1, colnum=N

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
1230.749628,1231.081643,1252.113717,0.247671,0.447169,0.265174,2.472573,0.203785,9.400602,254,-609.374814,260,0.203785


In [11]:
glmm_snr_s = glmer('SNR ~ Stain + (1|Experiment) + (1|ROI)', data=pl_df_measures, family='Gamma', link='log')
glmm_snr_s.set_factors(['Stain'])

# Convert estimates to odds
display(glmm_snr_s.fit(exponentiate=True, summary=True, conf_method='boot'))
display(glmm_snr_s.result_fit_stats)
stat_diagnostics(glmm_snr_s.data['fitted'], glmm_snr_s.data['resid'], glmm_snr_s.data['Experiment'])

GT(_tbl_data=shape: (7, 10)
┌────────────────┬────────────────┬───────────┬───────────┬───┬───────────┬──────┬─────────┬───────┐
│ rfx            ┆ param          ┆ estimate  ┆ conf_low  ┆ … ┆ z_stat    ┆ df   ┆ p_value ┆ stars │
│ ---            ┆ ---            ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---  ┆ ---     ┆ ---   │
│ str            ┆ str            ┆ f64       ┆ f64       ┆   ┆ f64       ┆ i64  ┆ str     ┆ str   │
╞════════════════╪════════════════╪═══════════╪═══════════╪═══╪═══════════╪══════╪═════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)    ┆ 0.097171  ┆ 0.145075  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ ROI-sd         ┆ (Intercept)    ┆ 0.063828  ┆ 0.106307  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Residual-sd    ┆ Observation    ┆ 0.204885  ┆ 0.218174  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ null           ┆ null      ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Fixed Effects: ┆ null           ┆ null      ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ (Intercept)    ┆ 10.109869 ┆ 1.78023   ┆ … ┆ 18.669942 ┆ 255  ┆ <.001   ┆ ***   │
│ null           ┆ StainPTA-stain ┆ 1.350062  ┆ -0.272897 ┆ … ┆ 2.297574  ┆ 255  ┆ 0.0224  ┆ *     │
│                ┆ ed             ┆           ┆           ┆   ┆           ┆      ┆         ┆       │
└────────────────┴────────────────┴───────────┴───────────┴───┴───────────┴──────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e75b0555e50>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e75b03c5430>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(SNR~Stain+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *260*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-610*  \n            AIC: *1230* | BIC: *1248*  \n            Residual error: *0.205*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(co

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
1230.436622,1230.672842,1248.24003,0.247381,0.443576,0.260682,2.483453,0.204885,9.462024,255,-610.218311,260,0.204885


In [12]:
ts.stats.anova(glmm_snr_snt, glmm_snr_sot, glmm_snr_s)

npar,AIC,BIC,logLik,-2*log(L),Chisq,Df,Pr(>Chisq)
f64,f64,f64,f64,f64,f64,f64,f64
5.0,1230.436622,1248.24003,-610.218311,1220.436622,null,null,null
6.0,1230.749628,1252.113717,-609.374814,1218.749628,1.686994,1.0,0.193998
7.0,1232.691493,1257.616264,-609.345746,1218.691493,0.058135,1.0,0.809469


In [13]:
display(glmm_snr_s.emmeans('Stain'))
display(glmm_snr_s.emmeans('Stain', contrasts='pairwise'))
display(glmm_snr_s.emmeans('Stain', contrasts='pairwise', p_adjust='bonf'))
display(glmm_snr_s.emmeans('Stain', contrasts='pairwise', p_adjust='mvt'))

Stain,response,SE,df,asymp_LCL,asymp_UCL
cat,f64,f64,f64,f64,f64
"""Iodine-stained""",10.109869,1.252779,inf,7.662789,13.338415
"""PTA-stained""",13.648951,1.278348,inf,11.069531,16.829427


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…",0.740707,0.096764,inf,0.573386,0.956854,1.0,-2.297574,0.021586


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…",0.740707,0.096764,inf,0.573386,0.956854,1.0,-2.297574,0.021586


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…",0.740707,0.096764,inf,0.573386,0.956854,1.0,-2.297574,0.021586


In [14]:
glmm_mean_snt = glmer('Mean ~ Stain * Tissue + (1|Experiment) + (1|ROI)', data=pl_df_measures, family='Gamma', link='log')
glmm_mean_snt.set_factors(['Stain', 'Tissue'])

# Convert estimates to odds
display(glmm_mean_snt.fit(exponentiate=True, summary=True, conf_method='boot'))
display(glmm_mean_snt.result_fit_stats)
stat_diagnostics(glmm_mean_snt.data['fitted'], glmm_mean_snt.data['resid'], glmm_mean_snt.data['Experiment'])

GT(_tbl_data=shape: (9, 10)
┌────────────────┬────────────────┬───────────┬───────────┬───┬──────────┬──────┬──────────┬───────┐
│ rfx            ┆ param          ┆ estimate  ┆ conf_low  ┆ … ┆ z_stat   ┆ df   ┆ p_value  ┆ stars │
│ ---            ┆ ---            ┆ ---       ┆ ---       ┆   ┆ ---      ┆ ---  ┆ ---      ┆ ---   │
│ str            ┆ str            ┆ f64       ┆ f64       ┆   ┆ f64      ┆ i64  ┆ str      ┆ str   │
╞════════════════╪════════════════╪═══════════╪═══════════╪═══╪══════════╪══════╪══════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)    ┆ 0.059344  ┆ 0.127053  ┆ … ┆ null     ┆ null ┆ null     ┆ null  │
│ ROI-sd         ┆ (Intercept)    ┆ 0.047144  ┆ 0.10124   ┆ … ┆ null     ┆ null ┆ null     ┆ null  │
│ Residual-sd    ┆ Observation    ┆ 0.124929  ┆ 0.148722  ┆ … ┆ null     ┆ null ┆ null     ┆ null  │
│ null           ┆ null           ┆ null      ┆ null      ┆ … ┆ null     ┆ null ┆ null     ┆ null  │
│ Fixed Effects: ┆ null           ┆ null      ┆ null      ┆ … ┆ null     ┆ null ┆ null     ┆ null  │
│ null           ┆ (Intercept)    ┆ 94.281302 ┆ 3.98816   ┆ … ┆ 52.31388 ┆ 253  ┆ <.001    ┆ ***   │
│ null           ┆ StainPTA-stain ┆ 1.296839  ┆ -0.312518 ┆ … ┆ 3.04274  ┆ 253  ┆ 0.002591 ┆ **    │
│                ┆ ed             ┆           ┆           ┆   ┆          ┆      ┆          ┆       │
│ null           ┆ TissueMuscle-s ┆ 1.140207  ┆ 0.074673  ┆ … ┆ 5.026837 ┆ 253  ┆ <.001    ┆ ***   │
│                ┆ erosal         ┆           ┆           ┆   ┆          ┆      ┆          ┆       │
│ null           ┆ StainPTA-stain ┆ 1.053568  ┆ -0.011919 ┆ … ┆ 1.66335  ┆ 253  ┆ 0.09748  ┆ .     │
│                ┆ ed:TissueMuscl ┆           ┆           ┆   ┆          ┆      ┆          ┆       │
│                ┆ e-…            ┆           ┆           ┆   ┆          ┆      ┆          ┆       │
└────────────────┴────────────────┴───────────┴───────────┴───┴──────────┴──────┴──────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e75b03c5b50>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e75b044c770>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(Mean~Stain*Tissue+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *260*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-1080*  \n            AIC: *2174* | BIC: *2199*  \n            Residual error: *0.125*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, c

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
2174.624129,2175.068574,2199.548901,0.270563,0.662584,0.537429,14.642606,0.124929,3.546964,253,-1080.312065,260,0.124929


In [15]:
glmm_mean_sot = glmer('Mean ~ Stain + Tissue + (1|Experiment) + (1|ROI)', data=pl_df_measures, family='Gamma', link='log')
glmm_mean_sot.set_factors(['Stain', 'Tissue'])

# Convert estimates to odds
display(glmm_mean_sot.fit(exponentiate=True, summary=True, conf_method='boot'))
display(glmm_mean_sot.result_fit_stats)
stat_diagnostics(glmm_mean_sot.data['fitted'], glmm_mean_sot.data['resid'], glmm_mean_sot.data['Experiment'])

GT(_tbl_data=shape: (8, 10)
┌────────────────┬────────────────┬───────────┬───────────┬───┬───────────┬──────┬─────────┬───────┐
│ rfx            ┆ param          ┆ estimate  ┆ conf_low  ┆ … ┆ z_stat    ┆ df   ┆ p_value ┆ stars │
│ ---            ┆ ---            ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---  ┆ ---     ┆ ---   │
│ str            ┆ str            ┆ f64       ┆ f64       ┆   ┆ f64       ┆ i64  ┆ str     ┆ str   │
╞════════════════╪════════════════╪═══════════╪═══════════╪═══╪═══════════╪══════╪═════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)    ┆ 0.05942   ┆ 0.124153  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ ROI-sd         ┆ (Intercept)    ┆ 0.047301  ┆ 0.102232  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Residual-sd    ┆ Observation    ┆ 0.125662  ┆ 0.149556  ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ null           ┆ null      ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ Fixed Effects: ┆ null           ┆ null      ┆ null      ┆ … ┆ null      ┆ null ┆ null    ┆ null  │
│ null           ┆ (Intercept)    ┆ 92.611105 ┆ 4.005765  ┆ … ┆ 52.602625 ┆ 254  ┆ <.001   ┆ ***   │
│ null           ┆ StainPTA-stain ┆ 1.330895  ┆ -0.285428 ┆ … ┆ 3.410554  ┆ 254  ┆ <.001   ┆ ***   │
│                ┆ ed             ┆           ┆           ┆   ┆           ┆      ┆         ┆       │
│ null           ┆ TissueMuscle-s ┆ 1.182137  ┆ 0.137092  ┆ … ┆ 11.483145 ┆ 254  ┆ <.001   ┆ ***   │
│                ┆ erosal         ┆           ┆           ┆   ┆           ┆      ┆         ┆       │
└────────────────┴────────────────┴───────────┴───────────┴───┴───────────┴──────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e75b0275580>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e75b02ebfe0>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(Mean~Stain+Tissue+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *260*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-1081*  \n            AIC: *2175* | BIC: *2196*  \n            Residual error: *0.126*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=1, colnum

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
2175.375478,2175.707493,2196.739567,0.269086,0.658918,0.533349,14.697236,0.125662,3.586407,254,-1081.687739,260,0.125662


In [16]:
ts.stats.anova(glmm_mean_snt, glmm_mean_sot)

npar,AIC,BIC,logLik,-2*log(L),Chisq,Df,Pr(>Chisq)
f64,f64,f64,f64,f64,f64,f64,f64
6.0,2175.375478,2196.739567,-1081.687739,2163.375478,null,null,null
7.0,2174.624129,2199.548901,-1080.312065,2160.624129,2.751348,1.0,0.097172


In [17]:
display(glmm_mean_sot.emmeans('Stain'))
display(glmm_mean_sot.emmeans('Stain', contrasts='pairwise', p_adjust='bonf'))
display(glmm_mean_sot.emmeans('Stain', by='Tissue', contrasts='pairwise', p_adjust='bonf'))

display(glmm_mean_sot.emmeans('Tissue'))
display(glmm_mean_sot.emmeans('Tissue', contrasts='pairwise', p_adjust='bonf'))
display(glmm_mean_sot.emmeans('Tissue', by='Stain', contrasts='pairwise', p_adjust='bonf'))

Stain,response,SE,df,asymp_LCL,asymp_UCL
cat,f64,f64,f64,f64,f64
"""Iodine-stained""",100.692464,8.638442,inf,83.113141,121.990001
"""PTA-stained""",134.011103,9.149692,inf,115.033779,156.11915


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…",0.751374,0.062976,inf,0.637549,0.885521,1.0,-3.410554,0.000648


contrast,Tissue,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…","""Mucosal-submucosal""",0.751374,0.062976,inf,0.637549,0.885521,1.0,-3.410554,0.000648
"""(Iodine-stained) / (PTA-staine…","""Muscle-serosal""",0.751374,0.062976,inf,0.637549,0.885521,1.0,-3.410554,0.000648


Tissue,response,SE,df,asymp_LCL,asymp_UCL
cat,f64,f64,f64,f64,f64
"""Mucosal-submucosal""",106.840269,7.011657,inf,92.255378,123.730923
"""Muscle-serosal""",126.299833,8.29008,inf,109.055906,146.270371


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Mucosal-submucosal) / (Muscle…",0.845926,0.012326,inf,0.822108,0.870433,1.0,-11.483145,1.6034e-30


contrast,Stain,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Mucosal-submucosal) / (Muscle…","""Iodine-stained""",0.845926,0.012326,inf,0.822108,0.870433,1.0,-11.483145,1.6034e-30
"""(Mucosal-submucosal) / (Muscle…","""PTA-stained""",0.845926,0.012326,inf,0.822108,0.870433,1.0,-11.483145,1.6034e-30


In [18]:
pl_df_cnr = pl.DataFrame(df_cnr)
glmm_cnr_s_lg = glmer('CNR ~ Stain + (1|Experiment) + (1|ROI)', data=pl_df_cnr, family='Gamma', link='log')
glmm_cnr_s_lg.set_factors(['Stain'])

# Convert estimates to odds
display(glmm_cnr_s_lg.fit(exponentiate=True, summary=True, conf_method='boot'))
display(glmm_cnr_s_lg.result_fit_stats)
stat_diagnostics(glmm_cnr_s_lg.data['fitted'], glmm_cnr_s_lg.data['resid'], glmm_cnr_s_lg.data['Experiment'])

GT(_tbl_data=shape: (7, 10)
┌────────────────┬────────────────┬──────────┬───────────┬───┬───────────┬──────┬──────────┬───────┐
│ rfx            ┆ param          ┆ estimate ┆ conf_low  ┆ … ┆ z_stat    ┆ df   ┆ p_value  ┆ stars │
│ ---            ┆ ---            ┆ ---      ┆ ---       ┆   ┆ ---       ┆ ---  ┆ ---      ┆ ---   │
│ str            ┆ str            ┆ f64      ┆ f64       ┆   ┆ f64       ┆ i64  ┆ str      ┆ str   │
╞════════════════╪════════════════╪══════════╪═══════════╪═══╪═══════════╪══════╪══════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)    ┆ 0.233652 ┆ 0.145577  ┆ … ┆ null      ┆ null ┆ null     ┆ null  │
│ ROI-sd         ┆ (Intercept)    ┆ 0.290188 ┆ 0.182995  ┆ … ┆ null      ┆ null ┆ null     ┆ null  │
│ Residual-sd    ┆ Observation    ┆ 0.652098 ┆ 0.572074  ┆ … ┆ null      ┆ null ┆ null     ┆ null  │
│ null           ┆ null           ┆ null     ┆ null      ┆ … ┆ null      ┆ null ┆ null     ┆ null  │
│ Fixed Effects: ┆ null           ┆ null     ┆ null      ┆ … ┆ null      ┆ null ┆ null     ┆ null  │
│ null           ┆ (Intercept)    ┆ 0.835821 ┆ -0.725737 ┆ … ┆ -0.733458 ┆ 125  ┆ 0.4647   ┆       │
│ null           ┆ StainPTA-stain ┆ 2.033532 ┆ 0.216377  ┆ … ┆ 2.921804  ┆ 125  ┆ 0.004131 ┆ **    │
│                ┆ ed             ┆          ┆           ┆   ┆           ┆      ┆          ┆       │
└────────────────┴────────────────┴──────────┴───────────┴───┴───────────┴──────┴──────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e75b02ebb90>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e75b02e89e0>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(CNR~Stain+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *130*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-175*  \n            AIC: *361* | BIC: *376*  \n            Residual error: *0.652*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(colu

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
361.796806,362.280677,376.134479,0.281468,0.410702,0.179858,1.012601,0.652098,72.863983,125,-175.898403,130,0.652098


In [19]:
display(glmm_cnr_s_lg.emmeans('Stain'))
display(glmm_cnr_s_lg.emmeans('Stain', contrasts='pairwise', p_adjust='bonf'))

Stain,response,SE,df,asymp_LCL,asymp_UCL
cat,f64,f64,f64,f64,f64
"""Iodine-stained""",0.835821,0.20437,inf,0.483747,1.444134
"""PTA-stained""",1.699668,0.326601,inf,1.105923,2.612181


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…",0.491755,0.119459,inf,0.305473,0.791636,1.0,-2.921804,0.00348


In [20]:
OFFSET = 2
df_cnr['CNR'] = df_cnr['CNR'] + OFFSET
pl_df_cnr = pl.DataFrame(df_cnr)
glmm_cnr_s = glmer('CNR ~ Stain + (1|Experiment) + (1|ROI)', data=pl_df_cnr, family='Gamma', link='log')
glmm_cnr_s.set_factors(['Stain'])

# Convert estimates to odds
display(glmm_cnr_s.fit(exponentiate=True, summary=True, conf_method='boot'))
display(glmm_cnr_s.result_fit_stats)
stat_diagnostics(glmm_cnr_s.data['fitted'], glmm_cnr_s.data['resid'], glmm_cnr_s.data['Experiment'])

GT(_tbl_data=shape: (7, 10)
┌────────────────┬──────────────────┬──────────┬───────────┬───┬──────────┬──────┬─────────┬───────┐
│ rfx            ┆ param            ┆ estimate ┆ conf_low  ┆ … ┆ z_stat   ┆ df   ┆ p_value ┆ stars │
│ ---            ┆ ---              ┆ ---      ┆ ---       ┆   ┆ ---      ┆ ---  ┆ ---     ┆ ---   │
│ str            ┆ str              ┆ f64      ┆ f64       ┆   ┆ f64      ┆ i64  ┆ str     ┆ str   │
╞════════════════╪══════════════════╪══════════╪═══════════╪═══╪══════════╪══════╪═════════╪═══════╡
│ Experiment-sd  ┆ (Intercept)      ┆ 0.100032 ┆ 0.159401  ┆ … ┆ null     ┆ null ┆ null    ┆ null  │
│ ROI-sd         ┆ (Intercept)      ┆ 0.1204   ┆ 0.163251  ┆ … ┆ null     ┆ null ┆ null    ┆ null  │
│ Residual-sd    ┆ Observation      ┆ 0.276477 ┆ 0.279062  ┆ … ┆ null     ┆ null ┆ null    ┆ null  │
│ null           ┆ null             ┆ null     ┆ null      ┆ … ┆ null     ┆ null ┆ null    ┆ null  │
│ Fixed Effects: ┆ null             ┆ null     ┆ null      ┆ … ┆ null     ┆ null ┆ null    ┆ null  │
│ null           ┆ (Intercept)      ┆ 2.92331  ┆ 0.551869  ┆ … ┆ 9.108996 ┆ 125  ┆ <.001   ┆ ***   │
│ null           ┆ StainPTA-stained ┆ 1.293474 ┆ -0.179195 ┆ … ┆ 2.469149 ┆ 125  ┆ 0.01489 ┆ *     │
└────────────────┴──────────────────┴──────────┴───────────┴───┴──────────┴──────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x7e75b015ce90>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='z_stat', type=<ColInfoTypeEnum.default: 1>, column_label='Z-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='df', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7e75b015d5e0>, _spanners=Spanners([]), _heading=Heading(title='Formula: glmer(CNR~Stain+(1|Experiment)+(1|ROI))', subtitle=Md(text='Family: *Gamma (link: *log*)*  \n            Number of observations: *130*  \n            Confidence intervals: *boot*   \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-176*  \n            AIC: *363* | BIC: *378*  \n            Residual error: *0.276*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None, mask=None), grpname=None, colname='param', rownum=2, colnum=None, styles=[C

AIC,AICc,BIC,ICC,R2_conditional,R2_marginal,RMSE,Sigma,deviance,df_residual,logLik,nobs,sigma
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64,f64
363.858411,364.342282,378.196083,0.249615,0.344536,0.126497,1.01738,0.276477,7.989181,125,-176.929206,130,0.276477


In [21]:
display(glmm_cnr_s.emmeans('Stain'))
display(glmm_cnr_s.emmeans('Stain', contrasts='pairwise', p_adjust='bonf'))

Stain,response,SE,df,asymp_LCL,asymp_UCL
cat,f64,f64,f64,f64,f64
"""Iodine-stained""",2.92331,0.344262,inf,2.246423,3.804156
"""PTA-stained""",3.781226,0.372421,inf,3.033669,4.712996


contrast,ratio,SE,df,asymp_LCL,asymp_UCL,null,z_ratio,p_value
cat,f64,f64,f64,f64,f64,f64,f64,f64
"""(Iodine-stained) / (PTA-staine…",0.773112,0.080573,inf,0.630276,0.948317,1.0,-2.469149,0.013543


In [22]:
def add_pvalue_annotation(fig, df, y_field, x_field, x_field_values, group_field=None, group_values=None, y_range=[1.01,1.04], 
                          text_offset=0.12, subplot=None, pvalue_th=0.05, color='dimgrey', text_size=14, pbar_offset=[0.0, 0.0]):
    # pbar_offset is [length of the bar, height of the bar]
    
    bar_xcoord_map = {x: idx for idx, x in enumerate(x_field_values)}
    
    # Using one-way ANOVA
    if group_field:
        ## for bars, there is a direct mapping from the bar number to 0, 1, 2...
        for x_field_value in x_field_values:
            pvalue = 0.1
            # print(f"P-value of {y_field} between two {group_field}s in {x_field_value}: {pvalue:.4f} - {'Not significant' if pvalue > pvalue_th else 'Significant'}")
            x_coord = bar_xcoord_map[x_field_value]
            fig = draw_statistical_significance(fig, y_range, pvalue_th, color, pvalue, 
                                                x_range=[x_coord-0.2, x_coord+0.2], 
                                                subplot=subplot, 
                                                text_offset=text_offset, text_size=text_size,
                                                pbar_offset=pbar_offset
                                               )
    else:
        pvalue = 0.1
        # print(f"P-value of {y_field} between two {x_field}s: {pvalue:.4f} - {'Not significant' if pvalue > pvalue_th else 'Significant'}")
        x_start = bar_xcoord_map[x_field_values[0]]
        x_end = bar_xcoord_map[x_field_values[1]]
        fig = draw_statistical_significance(fig, y_range, pvalue_th, color, pvalue, 
                                            x_range=[x_start, x_end], 
                                            subplot=subplot, 
                                            text_offset=text_offset, text_size=text_size,
                                            pbar_offset=pbar_offset
                                           )

    return fig

def draw_statistical_significance(fig, y_range, pvalue_th, color, pvalue, x_range, subplot, text_offset, text_size, pbar_offset):
    subplot_str = '' if (subplot==None or subplot==1 or subplot==0) else str(subplot)

    if pvalue >= pvalue_th:
        symbol = f'ns<br>p={round(pvalue, 3)}'
    elif pvalue >= 0.01: 
        symbol = '*'
    elif pvalue >= 0.001:
        symbol = '**'
    else:
        symbol = '***'

    x0 = x_range[0] - pbar_offset[0]
    x1 = x_range[1] + pbar_offset[0]
    y0 = y_range[0] - pbar_offset[1]
    y1 = y_range[1] + pbar_offset[1]

    # 1st vertical line
    fig.add_shape(type="line",
        xref=f"x{subplot_str}", yref=f"y{subplot_str} domain",
        # xref="x", yref="paper",
        x0=x0, y0=y0, x1=x0, y1=y1,
        line=dict(
            color=color,
            width=2,
        )
    )
    #  Horizontal line (long line)
    fig.add_shape(type="line",
        xref=f"x{subplot_str}", yref=f"y{subplot_str} domain",
        x0=x0, y0=y1, x1=x1, y1=y1,
        line=dict(
            color=color,
            width=2,
        )
    )
    # 2nd vertical line
    fig.add_shape(type="line",
        xref=f"x{subplot_str}", yref=f"y{subplot_str} domain",
        x0=x1, y0=y1, x1=x1, y1=y0,
        line=dict(
            color=color,
            width=2,
        )
    )
    # add text at the correct x, y coordinates
    # fig.add_annotation(dict(
    #     xref=f"x{subplot_str}", yref=f"y{subplot_str} domain",
    #     x=x_range[0]+((x_range[1]-x_range[0])/2),
    #     y=y_range[1]+text_offset,
    #     text=symbol,
    #     font=dict(color="dimgrey",size=text_size),
    #     textangle=0,
    #     showarrow=False,
    # ))

    return fig

In [27]:
fig = make_subplots(rows=1, cols=3, shared_yaxes=False, 
                    subplot_titles=['(a) Signal-to-noise ratio - SNR', '(b) Contrast-to-noise ratio - CNR', '(c) Mean intensity'], 
                    horizontal_spacing=0.10 )

def populate_subplot(fig, df, measure, col_no, offset=0):    
    for i, stain in enumerate(df['Stain'].unique()):
        fig.add_trace(go.Violin(
            x=df['Stain'][df['Stain']==stain],
            y=np.array(df[measure][df['Stain']==stain]) + offset if measure == 'CNR' else df[measure][df['Stain']==stain],
            line_color=COLOR_SCALE[i],
        ), row=1, col=col_no)

for col_no, measure in enumerate(['SNR', 'CNR', 'Mean']):
    if measure == 'CNR':
        populate_subplot(fig, df=df_cnr, measure=measure, col_no=col_no+1, offset=0)
    else:
        populate_subplot(fig, df=df_measures, measure=measure, col_no=col_no+1)

fig.update_annotations(font=dict(size=18), y=1.13)
fig.update_traces(box_visible=True, meanline_visible=True, scalemode='count', showlegend=False, width=0.85)
fig.update_layout(
    width=1000, height=300, margin=dict(t=60, b=60, l=10, r=10),
    yaxis=dict(range=[0, 31], dtick=5, title='Ratio', title_standoff = 10),
    yaxis2=dict(range=[0, 12.1], title='Ratio', title_standoff = 1),
    yaxis3=dict(range=[0, 252], title='Intensity', title_standoff = 7),
    font=dict(size=18)
)

for i, measure in enumerate(['SNR', 'CNR', 'Mean']):
    fig = add_pvalue_annotation(
        fig, df_measures, 
        y_field=measure, x_field='Stain', x_field_values=[STAIN_NAMES['I2KI'], STAIN_NAMES['PTA']],
        y_range=[1.0, 1.02],
        subplot=i+1,
        text_offset=0.09, pbar_offset=[0.0, 0.02]
    )

fig.write_image("quality_comparisons_of_stains_nonAnot.pdf")
fig.show()

In [24]:
ordered_rois = ["GC_Fundus1", "GC_Corpus", "GC_Antrum", "GC_Pylorus", "GC_LES", "GC_Fundus2", "LC_Corpus", "LC_Antrum", "LC_Pylorus", "LC_LES"]
df_pta = df_measures[df_measures['Stain'] == STAIN_NAMES['PTA']].reset_index(drop=True)
# Pick only the 9.08 um experiments
# df_pta = df_measures.loc[df_measures['Experiment'].isin(['RT09','RT10', 'RT11', 'RT12', 'RT13'])]
df_pta.head()

,Experiment,Tissue,ROI,Area,Mean,StdDev,Min,Max,Median,SNR,Stain
0,RT03,Muscle-serosal,GC_Antrum,0.102,146.944,8.872,117,169,148,16.562669,PTA-stained
1,RT03,Muscle-serosal,GC_Pylorus,0.101,200.120,11.526,161,223,202,17.362485,PTA-stained
2,RT03,Muscle-serosal,LC_Pylorus,0.104,160.530,11.530,119,186,161,13.922810,PTA-stained
3,RT03,Muscle-serosal,LC_LES,0.105,126.134,11.739,95,164,125,10.744868,PTA-stained
4,RT03,Muscle-serosal,GC_LES,0.101,154.200,10.317,123,179,155,14.946205,PTA-stained


In [25]:
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.2)

for i, roi in enumerate(ordered_rois):
    row = 2 if i>= 5 else 1
    fig.add_trace(go.Violin(x=df_pta['ROI'][(df_pta['Tissue'] == TISSUE_NAMES['Muscle']) & (df_pta['ROI'] == roi)],
                            y=df_pta['SNR'][(df_pta['Tissue'] == TISSUE_NAMES['Muscle']) & (df_pta['ROI'] == roi)],
                            legendgroup=TISSUE_NAMES['Muscle'], scalegroup=TISSUE_NAMES['Muscle'], name=TISSUE_NAMES['Muscle'],
                            side='negative',
                            line_color=COLOR_SCALE[0],
                            showlegend=True if i==0 else False),
                row=row, col=1
            )
    fig.add_trace(go.Violin(x=df_pta['ROI'][(df_pta['Tissue'] == TISSUE_NAMES['Mucosa']) & (df_pta['ROI'] == roi)],
                            y=df_pta['SNR'][(df_pta['Tissue'] == TISSUE_NAMES['Mucosa']) & (df_pta['ROI'] == roi)],
                            legendgroup=TISSUE_NAMES['Mucosa'], scalegroup=TISSUE_NAMES['Mucosa'], name=TISSUE_NAMES['Mucosa'],
                            side='positive',
                            line_color=COLOR_SCALE[1],
                            showlegend=True if i==0 else False),
                row=row, col=1
            )
    
fig = add_pvalue_annotation(
    fig, df_pta, 
    y_field='SNR', x_field='ROI', x_field_values=ordered_rois[:5],
    group_field='Tissue', group_values=['Muscle', 'Mucosa'],
    text_offset=0.12, subplot=1, text_size=14, pbar_offset=[0.07, 0.01]
)

fig = add_pvalue_annotation(
    fig, df_pta, 
    y_field='SNR', x_field='ROI', x_field_values=ordered_rois[5:],
    group_field='Tissue', group_values=['Muscle', 'Mucosa'],
    text_offset=0.12, subplot=2, text_size=14, pbar_offset=[0.07, 0.01]
)

fig.update_traces(meanline_visible=True, box_visible=True, scalemode='count')
fig.update_xaxes(automargin=True, range=[-0.5, 4.5])
fig.update_layout(
    violingap=0.001, violingroupgap=0, violinmode='overlay', 
    height=550, width=650, margin=dict(l=10, r=10, t=30, b=45), font=dict(size=18),
    yaxis=dict(range=[-2, 36], dtick=5, title='SNR', title_standoff = 5),
    yaxis2=dict(range=[-2, 31], dtick=5, title='SNR', title_standoff = 5),
    xaxis = dict(tickmode = 'array', tickvals = ordered_rois[:5], ticktext = [' '.join(roi.split('_')) for roi in ordered_rois[:5]]),
    xaxis2 = dict(tickmode = 'array', tickvals = ordered_rois[5:], ticktext = [' '.join(roi.split('_')) for roi in ordered_rois[5:]], title='ROI', title_standoff = 5),
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5, bgcolor="WhiteSmoke"),
)
fig.write_image("regional_snr.svg")
fig.show()